In [1]:
from openai import OpenAI
from pathlib import Path

key = open('openai.key').read().strip()
client = OpenAI(api_key=key, organization='org-l0QXnTWrsY221IPu8QIF1k1H', project='proj_bD68pqv8EEnCgfp2ujfsoJpJ')

In [2]:
txt_path = Path('public_agenda_june24_.txt')

In [3]:
with open('prompt.txt') as f:
    prompt = f.read()

In [4]:
assistant = client.beta.assistants.create(
    name="Technology Purchases",
    instructions=prompt,
    model='gpt-3.5-turbo',
    response_format={ "type": "json_object" }
)

In [5]:
results = []

with txt_path.open() as f:
    text = f.read()
    
    for i in range(0, len(text), 10000):
        chunk = text[i:i+10000]

        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt + '\n' + chunk,
                }
            ]
        )
    
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant.id
        )
    
        messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    
        message_content = messages[0].content[0].text
        
        results.append(message_content.value)
        print(message_content.value)

{
    "technology_purchases": [
        {
            "technology_system_name": "Versadex Records Management System",
            "developer_company_name": "",
            "deploying_company_name": "Versaterm Public Safety Inc.",
            "budget": "",
            "start_date": "",
            "end_date": ""
        },
        {
            "technology_system_name": "Courthouse Radio System",
            "developer_company_name": "",
            "deploying_company_name": "Infrastructure Ontario",
            "budget": "",
            "start_date": "",
            "end_date": ""
        }
    ]
}
{
  "technology_purchases": []
}
{}
{
    "technology_purchases": []
}
{
	"technology_purchases": []
}
{
    "technology_purchases": [
        {
            "technology_system_name": "",
            "developer_company_name": "",
            "deploying_company_name": "",
            "budget": "",
            "start_date": "",
            "end_date": ""
        }
    ]
}
{
	"technology_purchas

In [6]:
results

['{\n    "technology_purchases": [\n        {\n            "technology_system_name": "Versadex Records Management System",\n            "developer_company_name": "",\n            "deploying_company_name": "Versaterm Public Safety Inc.",\n            "budget": "",\n            "start_date": "",\n            "end_date": ""\n        },\n        {\n            "technology_system_name": "Courthouse Radio System",\n            "developer_company_name": "",\n            "deploying_company_name": "Infrastructure Ontario",\n            "budget": "",\n            "start_date": "",\n            "end_date": ""\n        }\n    ]\n}',
 '{\n  "technology_purchases": []\n}',
 '{}',
 '{\n    "technology_purchases": []\n}',
 '{\n\t"technology_purchases": []\n}',
 '{\n    "technology_purchases": [\n        {\n            "technology_system_name": "",\n            "developer_company_name": "",\n            "deploying_company_name": "",\n            "budget": "",\n            "start_date": "",\n           

In [8]:
import json

json_results = [json.loads(r) for r in results]

In [9]:
# Drop any lines where the length is zero or there is a key called "technology_purchases" but its corresponding value is empty
filtered_results = [r for r in json_results if len(r) > 0 and ('technology_purchases' not in r or len(r['technology_purchases']) > 0)]

In [14]:
import pandas as pd

final_results = []
for entry in filtered_results:
    if 'technology_purchases' in entry:
        final_results.extend(entry['technology_purchases'])
        
df = pd.DataFrame(final_results)

In [15]:
df

,technology_system_name,developer_company_name,deploying_company_name,budget,start_date,end_date
0,Versadex Records Management System,,Versaterm Public Safety Inc.,,,
1,Courthouse Radio System,,Infrastructure Ontario,,,
2,,,,,,
3,XYZ Surveillance System,TechCorp,DeployIt Ltd.,1200000,2023-05-01,2024-04-30
4,ABC Crime Analytics,DataInsight,,800000,,2024-07-14
5,XYZ Surveillance System,TechCorp,DeployIt Ltd.,1200000,2023-05-01,2024-04-30
6,ABC Crime Analytics,DataInsight,,800000,,2024-07-14
7,Public Safety Data Portal,,,0,,
8,Versadex Records Management System,Versaterm Public Safety Inc.,,4450000,2024-11-05,2026-11-04
9,New Courthouse Radio System,H.M.K.,,0,2023-03-26,2025-01-31
